_________________________________________________________________________________________

##CV ASSIGNMENT 2

###NAME : GABRIEL JOSHUA R

###REG NO. : 20MIA1100

_________________________________________________________________________________________

#OPTICAL FLOW :

It allows us to understand how objects in a scene are moving and changing over time. This information can be used for a variety of tasks, such as tracking moving objects, stabilizing video footage, or generating realistic visual effects. To estimate optical flow, computer vision algorithms typically rely on assumptions about the properties of the scene, such as brightness constancy and smoothness. These algorithms use techniques such as the Lucas-Kanade method or Horn-Schunck algorithm to compute the optical flow field from a sequence of images.

_________________________________________________________________________________________

###Optical Flow with Lucas-Kanade method

The Lucas-Kanade method is a local optical flow algorithm that assumes the motion of a pixel is constant over a small region around it. It computes the displacement vector for each pixel by solving a set of linear equations based on the gradient and spatiotemporal derivatives of the image. The algorithm works well for small displacements and small changes in illumination but can fail for large displacements or occlusions. It does not handle changes in scene geometry or camera motion.

In [ ]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow

In [ ]:
vdo = cv.VideoCapture("/content/car1.mp4")

In [ ]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

In [ ]:
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

In [ ]:
# Create some random colors
color = np.random.randint(0, 255, (100, 3))

In [ ]:
# Take first frame and find corners in it
ret, old_frame = vdo.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

In [ ]:
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret, frame = vdo.read()
    if not ret:
        print('end of frame')
        break
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)
    cv2_imshow(img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
cv.destroyAllWindows()

_________________________________________________________________________________________

##Gunnar-Farneback optical flow

Gunnar-Farneback optical flow is a dense optical flow algorithm that estimates the motion of all pixels between two frames. It computes the optical flow field by analyzing the polynomial expansion of the image brightness function. The algorithm uses a pyramidal approach to estimate the motion of pixels at different scales. It is computationally intensive but can handle large displacements and changes in illumination.

In [ ]:
vdo = cv.VideoCapture(cv.samples.findFile("/content/car1.mp4"))

In [ ]:
ret, frame1 = vdo.read()

In [ ]:
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

In [ ]:
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = vdo.read()
    if not ret:
        print('end of frame')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv2_imshow(bgr)
    prvs = next
cv.destroyAllWindows()

###INFERENCE :

Both the above methods is efficiently able to detect the relative motion of the car and the surroundings. Based on the memory requirement, Lucas-Kanade is better as it uses only the important features from the frames of video.

_________________________________________________________________________________________